In [1]:
"""
This notebook is designed to showcase how to build your own subnet, modeled after the 
Bittensor Text Prompting subnet. This subnet is designed to take in a list of messages
and a list of roles, and generate a completion based on the current state of the network.

To create a subnet, you must define three (3) main elements: 
- the protocol message format (synapse subclass)
- the miner (how to complete requests)
- the validator (how to validate requests)

Below is a simple implementation of the protocol, so you can see how it works.

#NOTE: You MUST implement "{field}_hash" for any required field in your synapse.

For example, if you have a `messages` field, there must also be a complementary
`messages_hash` field. This is used to ensure that the data is not tampered with
during transit. You can use the `required_hash_fields` property to specify which
fields are required for hash computation.

You can also use the `deserialize` method to specify how to deserialize the data
into a tensor. This is useful for converting strings into tensors, for example.

You must also define your priority and blacklist functions. See below:
"""


import bittensor as bt
import pydantic
import time
import torch
bt.trace()

from typing import List, Dict, Optional, Tuple, Union, Callable
# Subnet creator controls
class Prompting( bt.Synapse ):
    """
    Represents the core component of a Synapse for handling and controlling message prompting in a network.
    
    The Prompting synapse captures roles and messages, and can generate a completion based on the current state. 
    It also contains unique hashes for roles and messages to ensure integrity and uniqueness.
    
    Attributes:
        roles (List[str]): A list of roles associated with this synapse. Immutable post-instantiation.
        messages (List[str]): A list of messages associated with this synapse. Immutable post-instantiation.
        completion (str): A field to store the completion or result after processing.
        messages_hash (str): An immutable hash representing the messages in the prompting scenario.
        roles_hash (str): An immutable hash representing the roles in the prompting scenario.
    """
    class Config: validate_assignment = True
    def deserialize( self ): return self.completion

    roles: List[str] = pydantic.Field(..., allow_mutation=False)
    messages: List[str] = pydantic.Field(..., allow_mutation=False)
    completion: str = None

    messages_hash: str = pydantic.Field(
        "",
        title="Messages Hash",
        description="Hash of the messages in the Prompting scenario. Immutable.",
        allow_mutation=False,
    )

    roles_hash: str = pydantic.Field(
        "",
        title="Roles Hash",
        description="Hash of the roles in the Prompting scenario. Immutable.",
        allow_mutation=False,
    )

    @property
    def required_hash_fields(self) -> List[str]:
        """ Returns the list of fields that are essential for hash computation. """
        return ['messages']

def prompt( synapse: Prompting ) -> Prompting:
    """
    Process the provided synapse to generate a completion.

    Args:
        synapse (Prompting): The input synapse to be processed.

    Returns:
        Prompting: The updated synapse with a completion.
    """
    bt.logging.debug("In prompt!")
    synapse.completion = "I am a chatbot"
    return synapse

def blacklist( synapse: Prompting ) -> Tuple[bool, str]:
    """
    Determines if the provided synapse should be blacklisted.

    Args:
        synapse (Prompting): The input synapse to be evaluated.

    Returns:
        Tuple[bool, str]: A tuple containing a boolean that indicates whether the synapse is blacklisted,
                          and a string providing the reason.
    """
    return False, ""

def priority( synapse: Prompting ) -> float:
    """
    Determines the priority of the provided synapse.

    Args:
        synapse (Prompting): The input synapse to be evaluated.

    Returns:
        float: The priority value of the synapse, with higher values indicating higher priority.
    """
    return 0.0


In [2]:
# Create an Axon instance on port 8099.
axon = bt.axon(port=8099)

# Attach the forward, blacklist, and priority functions to the Axon.
# forward_fn: The function to handle forwarding logic.
# blacklist_fn: The function to determine if a request should be blacklisted.
# priority_fn: The function to determine the priority of the request.
axon.attach(
    forward_fn=prompt,
    blacklist_fn=blacklist,
    priority_fn=priority
)

# Start the Axon to begin listening for requests.
axon.start()

# Create a Dendrite instance to handle client-side communication.
d = bt.dendrite()

# Send a request to the Axon using the Dendrite, passing in a StreamPrompting instance with roles and messages.
# The response is awaited, as the Dendrite communicates asynchronously with the Axon.
resp = await d(
    [axon],
    Prompting(roles=["user"], messages=["hello, who are you?"]),
    deserialize=False
)

# The response object contains the result of the prompting operation.
resp

INFO:     Started server process [3088652]
INFO:     Waiting for application startup.
TRACE:    ASGI [1] Started scope={'type': 'lifespan', 'asgi': {'version': '3.0', 'spec_version': '2.0'}, 'state': {}}
TRACE:    ASGI [1] Receive {'type': 'lifespan.startup'}
TRACE:    ASGI [1] Send {'type': 'lifespan.startup.complete'}
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8099 (Press CTRL+C to quit)


2023-09-19 02:12:06.962 |      TRACE       | Pre-process synapse for request


TRACE:    127.0.0.1:43048 - HTTP connection made


2023-09-19 02:12:06.962 |      DEBUG       | dendrite | --> | 3837 B | Prompting | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 216.153.62.113:8099 | 0 | Success


TRACE:    127.0.0.1:43048 - ASGI [2] Started scope={'type': 'http', 'asgi': {'version': '3.0', 'spec_version': '2.3'}, 'http_version': '1.1', 'server': ('127.0.0.1', 8099), 'client': ('127.0.0.1', 43048), 'scheme': 'http', 'method': 'POST', 'root_path': '', 'path': '/Prompting', 'raw_path': b'/Prompting', 'query_string': b'', 'headers': '<...>', 'state': {}}


...

2023-09-19 02:12:06.987 |      TRACE       | Fill successful response      


TRACE:    127.0.0.1:43048 - ASGI [2] Completed


2023-09-19 02:12:06.992 |      TRACE       | Finally                       
2023-09-19 02:12:06.993 |      DEBUG       | axon     | --> | 805 B | Prompting | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 127.0.0.1:43048  | 200 | Success
2023-09-19 02:12:06.995 |      TRACE       | Non-streaming response detected.
2023-09-19 02:12:06.997 |      TRACE       | Postprocess server response   
2023-09-19 02:12:06.998 |      DEBUG       | dendrite | <-- | 4391 B | Prompting | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 216.153.62.113:8099 | 200 | Success


2023-09-19 02:12:06.998 |      DEBUG       | dendrite | <-- | 4391 B | Prompting | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 216.153.62.113:8099 | 200 | Success


TRACE:    127.0.0.1:43048 - HTTP connection lost
